In [8]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm

In [9]:
# Canada Dataframe
dfCa=pd.read_csv('../data/canada/listings.csv')
dfCa.shape

(13621, 18)

In [10]:
dfCa.sample(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
13065,704178062502328876,Exquisite Skyview Downtown Montreal Apartment,429982362,Harshita,NaN,Ville-Marie,45.491220,-73.582037,Entire home/apt,351,2,0,NaN,NaN,4,364,0,NaN
11095,637092613727921684,Spacious 2 BR Victorian Style Apartment,437190719,Miguel,NaN,Ville-Marie,45.490180,-73.578010,Entire home/apt,191,1,7,2022-09-09,3.50,2,22,7,NaN
11511,649364295276697470,Nomade MTL | Le Penfield | Modern Queen Room,227489124,Nomade MTL,NaN,Ville-Marie,45.497070,-73.585900,Private room,205,2,2,2022-07-04,0.70,42,309,2,NaN
13195,709509641384234191,Élégant 3 bedroom unit with patio,144016038,Pamela,NaN,Mercier-Hochelaga-Maisonneuve,45.544329,-73.551001,Entire home/apt,64,32,0,NaN,NaN,3,315,0,NaN
11819,659314153944331485,Lovely room in Little Italy.,466560343,MeganM,NaN,Rosemont-La Petite-Patrie,45.534890,-73.609142,Private room,50,1,4,2022-08-01,2.18,1,75,4,NaN
9960,542614111338075255,MONTREAL | Cosy Basement | 20 min DTOWN,439160268,Comfort,NaN,Villeray-Saint-Michel-Parc-Extension,45.553380,-73.611900,Entire home/apt,91,1,37,2022-08-27,4.81,3,71,37,NaN
8441,49436040,feel home in a gorgeous basement + a small bed...,107258060,Nima,NaN,Dollard-des-Ormeaux,45.488490,-73.818380,Private room,79,2,25,2022-07-20,2.39,2,133,25,NaN
7398,43574134,Luxurious Old Montreal 2BDR - Great Location!,3437394,Caroline,NaN,Ville-Marie,45.511540,-73.555510,Entire home/apt,225,15,0,NaN,NaN,1,0,0,NaN
4913,29136613,GATSBY✦LegalAirBnb✦Rooftop✦Plateau✦SelfCheckIn✦,219345292,Unique Stays,NaN,Le Plateau-Mont-Royal,45.517700,-73.572160,Entire home/apt,164,1,155,2022-09-05,3.27,9,106,23,289951
9441,53396620,Multiple Apartment in the same building,315956014,Edward,NaN,Ville-Marie,45.495330,-73.580440,Entire home/apt,1301,1,0,NaN,NaN,44,118,0,NaN


In [11]:
# Ireland Dataframe
dfIr=pd.read_csv('../data/ireland/listings.csv')
dfIr.shape

(7566, 18)

In [12]:
dfIr.sample(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
4388,34851952,1 double bedroom with shower in shared apartment,175096630,Redouane,NaN,Dn Laoghaire-Rathdown,53.27063,-6.20570,Private room,70,1,2,2019-06-09,0.05,1,0,0,NaN
4100,30948708,Large and comfortable double bedroom,127415415,Fabio,NaN,Dn Laoghaire-Rathdown,53.30352,-6.24736,Private room,75,30,12,2022-02-26,0.32,1,3,7,NaN
2059,18465694,Cosy Cabin with Stunning Sea Views,84288638,Toni,NaN,Fingal,53.39163,-6.09412,Entire home/apt,177,3,339,2022-09-08,5.26,1,178,57,NaN
2448,20399524,Quiet place near the city center and Phoenix park,65605451,Yannis,NaN,Dublin City,53.34170,-6.29541,Private room,30,5,1,2017-08-16,0.02,1,0,0,NaN
4170,32914568,Dynamic Bungalow (Airport-5 Mins Away),242454069,Luke,NaN,Fingal,53.41039,-6.23205,Entire home/apt,72,1,14,2019-07-28,0.33,1,0,0,NaN
2891,23138544,unique and beautiful loft home in central dublin,3663400,Mary,NaN,Dublin City,53.35684,-6.24495,Entire home/apt,50,2,8,2018-04-16,0.14,1,0,0,NaN
6042,54313076,Great Home in Lucan,102734317,Michael,NaN,South Dublin,53.35554,-6.43830,Entire home/apt,310,15,5,2022-05-03,0.68,11,144,5,NaN
4781,37736016,A Home from Home,1971126,Bego,NaN,South Dublin,53.30609,-6.33051,Private room,60,2,2,2019-11-20,0.06,1,0,0,NaN
3641,27546939,Nikki's .. 10 min drive from Dublin Airpprt,47854906,Nicola,NaN,Fingal,53.52247,-6.16914,Private room,200,1,54,2020-03-18,1.09,1,12,0,NaN
4569,35856841,"Luxury Serviced room,in unique private Residen...",268990098,Daniel,NaN,Dublin City,53.31737,-6.27334,Private room,120,27,21,2019-09-15,0.54,5,130,0,NaN


In [13]:
# Scotland Dataframe
dfSc=pd.read_csv('../data/scotland/listings.csv')
dfSc.shape

(7818, 18)

In [14]:
dfSc.sample(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
6482,641833022826478570,Lovely bedroom 15 min to city centre,460844252,Marcel,NaN,Carrick Knowe,55.93549,-3.27121,Private room,52,1,31,2022-09-07,10.33,3,283,31,NaN
5603,54287183,Standard Twin Ensuite at Hanover House Hotel,238865026,Jasmin,NaN,Hillside and Calton Hill,55.95919,-3.18294,Private room,119,1,0,NaN,NaN,3,288,0,NaN
4359,42189842,Edinburgh.Alexander Guest House.luxury double ...,333709083,Yu,NaN,"Blackford, West Mains and Mayfield Road",55.93072,-3.17260,Private room,180,1,3,2022-01-30,0.23,9,190,2,NaN
7195,675764775131505910,Modern 1 bedroom central apartment by the sea,3535509,Mel,NaN,Granton and Royston Mains,55.98040,-3.23648,Entire home/apt,93,8,1,2022-08-06,0.75,2,80,1,NaN
707,7092500,Edinburgh's vibrant seaside area - Portobello.,37152599,Eric,NaN,Joppa,55.95339,-3.10664,Entire home/apt,78,3,56,2020-01-02,0.65,1,1,0,NaN
1199,13404391,"ALTIDO Spacious 2BR Apt, moments from Princes St",76306561,David,NaN,"Old Town, Princes Street and Leith Street",55.95222,-3.19548,Entire home/apt,181,2,293,2022-09-04,3.87,1,307,27,NaN
2030,20123599,Quartermile Apartment with Private Balcony,143044388,Agus & Ruth,NaN,Meadows and Southside,55.94335,-3.19483,Entire home/apt,287,2,70,2022-09-10,1.15,3,0,13,NaN
805,7610689,"Quiet, attractive single room",17821476,Helen,NaN,Morningside,55.92970,-3.21033,Private room,50,2,273,2022-08-29,3.16,1,141,39,NaN
357,3316549,(1) Double room walk to Royal Mile,15916423,Christine,NaN,Abbeyhill,55.95463,-3.16748,Private room,35,21,66,2022-09-01,0.67,2,32,8,NaN
5511,53716694,Quiet Room 15 min from Old Town and Leith,185678203,Łukasz,NaN,West Pilton,55.97186,-3.23807,Private room,39,10,21,2022-08-28,2.41,3,99,21,NaN


In [15]:
# Spain Dataframe
dfSp=pd.read_csv('../data/spain/listings.csv')
dfSp.shape

(6494, 18)

In [16]:
dfSp.sample(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
2094,25834185,"Room FIBES, with pool during summer season",157835871,Africa,Este - Alcosa - Torreblanca,"Colores, Entreparques",37.398030,-5.923020,Private room,32,1,17,2020-10-18,0.34,3,0,0,VFT/SE01844
4231,47710581,Exclusive and comfortable apartment at the center,91039666,David,Casco Antiguo,San Vicente,37.396320,-5.998380,Entire home/apt,297,1,75,2022-09-14,4.38,33,264,55,VFT/SE/06991
6217,671484282631578260,Amplio apartamento reformado en pleno centro,135867871,Pickman Sala,Nervión,La Florida,37.387866,-5.983991,Entire home/apt,60,62,0,NaN,NaN,8,143,0,Exempt
6039,640340417644812894,Nice flat with Parking in the Heart of Seville,462181746,Marta Del Rocio,Casco Antiguo,Santa Catalina,37.391960,-5.986950,Entire home/apt,97,2,27,2022-09-21,7.57,1,315,27,VFT/SE/01411
3072,36628735,RentalSevilla Apartamento en Santa Cruz - Parking,170512874,Rosalía,Casco Antiguo,San Bartolomé,37.387590,-5.985990,Entire home/apt,131,1,43,2022-08-24,1.11,105,309,6,VFT/SE/04372
2124,26388460,La Terraza de Triana,31042641,Jesús,Triana,Triana Oeste,37.383150,-6.008980,Entire home/apt,55,2,14,2020-03-12,0.28,1,0,0,VTF/SE/03910
1551,21250878,Apartamento en el corazón de Sevilla,15779574,Tatiana,Casco Antiguo,Santa Cruz,37.386320,-5.991040,Entire home/apt,70,2,40,2022-06-14,0.66,4,166,8,VFT/SE/3179
2053,24691301,Nervion Sevilla,136963259,Inma,Nervión,Huerta del Pilar,37.385270,-5.977090,Private room,50,3,3,2019-10-09,0.06,1,362,0,Exempt
4089,45525264,Monsalves 39 AT5,367492927,At,Casco Antiguo,"Encarnación, Regina",37.393170,-5.998570,Entire home/apt,75,1,15,2022-09-08,1.24,7,265,15,CTC-2020163809
379,3292591,Attic in the heart of Sevilla,13492532,Javier,Casco Antiguo,Santa Catalina,37.391970,-5.986140,Entire home/apt,61,2,299,2022-09-10,3.12,1,74,37,VFT/SE/00469


In [17]:
# Switzerland Dataframe
dfSw=pd.read_csv('../data/switzerland/listings.csv')
dfSw.shape

(3370, 18)

In [18]:
dfSw.sample(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
1633,50539017,Chambre Grand Lit Standard,349323361,Yannick,NaN,Commune de Genève,46.21601,6.14998,Entire home/apt,233,1,19,2022-09-03,1.39,5,304,15,NaN
644,19867995,Luminous room near Cornavin Station,65591301,David,NaN,Commune de Genève,46.20937,6.13477,Private room,49,3,4,2021-07-27,0.06,1,0,0,NaN
1751,52684438,Great apart Geneva center Cornavin Manor,118735300,Grant,NaN,Commune de Genève,46.20835,6.14378,Entire home/apt,120,1,87,2022-09-08,7.86,16,311,87,NaN
2867,33307999,HOTEL ORNEX DEUX LITS SEPARES,244299841,Nevada,NaN,Collex-Bossy,46.27787,6.09141,Hotel room,69,1,0,NaN,NaN,14,113,0,NaN
3193,685428757872526063,Annemasse superbe appartement T3/bis de 77m2,472926718,Marlene,NaN,Puplinge,46.18750,6.24140,Entire home/apt,89,5,0,NaN,NaN,2,365,0,NaN
2688,13143770,Top floor loft apartment with a view,73286317,Daniela & Francisco,NaN,Céligny,46.35016,6.16609,Entire home/apt,120,2,21,2022-08-15,0.32,1,89,1,NaN
686,21359870,"Appart.fully furnished 4,5 r Geneva, for long ...",4340068,Alessia,NaN,Commune de Genève,46.20604,6.11307,Entire home/apt,135,90,0,NaN,NaN,3,295,0,NaN
2200,690394736539303830,Cozy studio in the center of Geneva,38168263,Youssef,NaN,Commune de Genève,46.19384,6.14440,Entire home/apt,110,2,3,2022-09-09,3.00,4,340,3,NaN
3120,22336288,Spacious flat next to Geneva,142943294,Caroline,NaN,Collex-Bossy,46.25965,6.11059,Entire home/apt,105,275,24,2022-08-12,0.44,1,244,7,NaN
293,5009111,Lovely apartment with Mountain and Lake View,25557589,Olga,NaN,Commune de Genève,46.21431,6.14708,Entire home/apt,95,1,23,2020-03-12,0.52,1,0,0,NaN


In [19]:
# Thailand Dataframe
dfTh=pd.read_csv('../data/thailand/listings.csv')
dfTh.shape

(16640, 18)

In [20]:
dfTh.sample(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
15032,670880794679672039,Studio Twin Bed - ( Room Only ),97269224,Ken,NaN,Bang Rak,13.718497,100.518323,Private room,1071,1,0,NaN,NaN,5,365,0,NaN
2904,16178443,3-Bangkok(Woman) by SUPER HOST!,73922592,ﾊﾟｯﾄﾞﾀｲ,NaN,Bang Na,13.667980,100.616310,Entire home/apt,740,90,20,2020-03-31,0.31,4,363,0,NaN
2805,15749282,Puengwhan Ari Soi 5,101793824,朝霞,NaN,Phaya Thai,13.782470,100.540920,Private room,3800,1,1,2016-10-31,0.01,1,90,0,NaN
1169,7537579,"Executive Accommodation Pool Gym WiFI, Near BTS",19266319,Alex,NaN,Vadhana,13.715260,100.593960,Entire home/apt,1110,14,379,2022-08-26,4.38,34,303,8,NaN
13973,575105572947067597,♥Sasi 301 : Deluxe Room @MRT Huai-Khwang/Free ...,240603715,Sasikarn,NaN,Huai Khwang,13.768830,100.577850,Private room,1290,1,0,NaN,NaN,31,263,0,NaN
136,959254,"Deluxe Condo, Nana, Pool/GYM/Sauna",5153476,Natcha,NaN,Khlong Toei,13.715160,100.568060,Entire home/apt,1400,30,0,NaN,NaN,1,365,0,NaN
16586,618097086280398538,Stay with me //Baby Blue// @Poppular condo C5,29074575,Janee,NaN,Lak Si,13.912870,100.554300,Entire home/apt,700,1,8,2022-09-04,1.74,5,174,8,NaN
8834,35698483,疫情特惠～全面消毒-安心入住-中国人网红拉差达区域代购街/超大漂亮无边际泳池/火车夜市/拉差达夜市,71027407,E Thai Home,NaN,Huai Khwang,13.770800,100.582830,Entire home/apt,633,30,5,2021-01-29,0.13,21,68,0,NaN
5842,26611808,Family Room #6: 2 bed / 1.5 bath,180473956,Abe,NaN,Khlong Toei,13.721540,100.579280,Private room,999,2,51,2022-09-06,1.02,8,332,6,NaN
16252,711830970106562793,Compact cozy sunny room in Samutprakarn.,201946118,Leena,NaN,Bang Na,13.645426,100.636478,Private room,1120,1,0,NaN,NaN,2,365,0,NaN


In [21]:
# Turkey Dataframe
dfTu=pd.read_csv('../data/turkey/listings.csv')
dfTu.shape

(33259, 18)

In [22]:
dfTu.sample(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
27439,676440333107128753,"Havuz,sauna,fitnes,cafe sosyal alanlar mevcut.",470879037,Ergün,NaN,Basaksehir,41.11320,28.76852,Entire home/apt,26250,1,0,NaN,NaN,1,365,0,NaN
16996,53665269,Superıor Room,244844459,Cihat,NaN,Beyoglu,41.02552,28.97900,Private room,4240,365,1,2021-12-22,0.11,4,365,1,NaN
23218,633735866478289960,Junior Suite near to the Blue Mosque,289796182,Ramazan,NaN,Fatih,41.00377,28.97323,Private room,2147,2,1,2022-08-25,0.91,9,352,1,NaN
20268,588390164628890728,Spacious Panoramic Penthouse w/ Terrace & AC,39240214,Turaç,NaN,Beyoglu,41.03486,28.99267,Entire home/apt,2954,1,33,2022-09-11,5.69,1,321,33,NaN
28979,690318796978925221,Enjoy new cleanest apartment 1 year old. 2+1,461412969,Burak,NaN,Eyup,41.05657,28.93163,Entire home/apt,650,2,0,NaN,NaN,3,345,0,NaN
21906,615544543682957900,Modern 1Room Apartment 80,325818489,Khalid,NaN,Kucukcekmece,41.00133,28.80402,Entire home/apt,789,5,0,NaN,NaN,9,365,0,NaN
22743,625031962610095766,KADIKÖY MERKEZDE FENERBAHÇE STADYUM YANI LÜX,458346132,Inan,NaN,Kadikoy,40.98876,29.03290,Private room,1464,1,0,NaN,NaN,5,365,0,NaN
22484,621514851634496426,A room for one person..,388265465,Bouchra,NaN,Cekmekoy,41.01411,29.20154,Private room,258,1,0,NaN,NaN,1,365,0,NaN
18423,546066089409307303,1+0 Stüdyo Sabiha Gökçen 3km Viaport AVM 1km,429112148,Recep,NaN,Pendik,40.94139,29.30477,Shared room,216,1,0,NaN,NaN,1,363,0,NaN
16106,53040032,"Great location in Bostanci,free parking!",210780350,Selin,NaN,Kadikoy,40.95629,29.09377,Entire home/apt,1400,7,8,2022-05-14,0.82,1,109,8,NaN
